In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Otwieranie pliku


In [2]:
db = pd.read_csv('../db/out/wta_std.csv')

db['date'] = pd.to_datetime(db['date'], format='%Y-%m-%d')

db.dropna()

C:\Users\march\AppData\Local\Temp\ipykernel_17596\4103150540.py:1: DtypeWarning: Columns (10,28) have mixed types. Specify dtype option on import or set low_memory=False.
  db = pd.read_csv('../db/out/wta_std.csv')


,Unnamed: 0,match_id,date,surface,tourney_level,draw_size,y,name_P1,hand_P1,entry_P1,...,2ndWon_P2,SvGms_P2,bpSaved_P2,bpFaced_P2,elo_P1,elo_P2,glicko_P1,glicko_P2,RD_P1,RD_P2
22,22,65162,2023-08-28,Hard,G,128,0,Jennifer Brady,R,WC,...,16.0,13.0,4.0,7.0,1500.0,1484.0,1500.000000,1635.111824,350.000000,274.291813
23,23,65161,2023-08-28,Hard,G,128,0,Bernarda Pera,L,WC,...,11.0,14.0,6.0,11.0,1500.0,1500.0,1500.000000,1750.459722,350.000000,246.289317
24,24,65160,2023-08-28,Hard,G,128,0,Kaja Juvan,R,Q,...,12.0,7.0,0.0,0.0,1500.0,1468.0,1500.000000,1385.905907,350.000000,285.513318
25,25,65158,2023-08-28,Hard,G,128,1,Clara Burel,R,Q,...,8.0,9.0,5.0,10.0,1484.0,1500.0,1823.957565,1500.000000,294.941965,350.000000
26,26,65156,2023-08-28,Hard,G,128,1,Daria Kasatkina,R,Q,...,13.0,14.0,2.0,9.0,1484.0,1500.0,1407.277490,1500.000000,290.282388,350.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36787,36787,9799,2003-05-26,Clay,G,128,1,Barbara Schett,R,Q,...,14.0,13.0,12.0,18.0,876.0,1628.0,1440.517960,1425.638478,112.288545,43.587195
36788,36788,9798,2003-05-26,Clay,G,128,0,Angelique Widjaja,R,Q,...,15.0,16.0,4.0,12.0,1548.0,1196.0,1547.893134,1351.043465,154.267756,134.369976
36789,36789,9797,2003-05-26,Clay,G,128,0,Barbara Rittner,R,Q,...,10.0,8.0,3.0,4.0,1244.0,1228.0,1569.936236,1505.054619,192.427705,14.132388
36790,36790,9162,2003-04-26,Clay,D,4,1,Amelie Mauresmo,R,Q,...,16.0,10.0,3.0,6.0,1580.0,1612.0,1460.884983,1419.909833,23.988320,71.060777


In [3]:
from features import basicfeatureCreator
basic_stats = basicfeatureCreator(db)

In [4]:
from vsFeatures import vsfeatureCreator

vsstats = vsfeatureCreator(db)

pvp:  19287
common:  12438
missing:  5067


In [5]:
mean_pvp = pd.merge(
    basic_stats, vsstats, on=['match_id'], how='inner', suffixes=('', '_VS'))

In [6]:
mean_pvp.drop(
    ['draw_size', 'tourney_level', 'surface', 'hand_P1', 'hand_P2'], inplace=True, axis=1)

In [7]:
mean_pvp.to_csv('../db/out/wta_mp.csv', index=False)

In [8]:
df = mean_pvp

testp1 = df.filter(regex='_P1_VS$', axis=1)

testp2 = df.filter(regex='_P2_VS$', axis=1)


feats = list(zip(testp1.columns, testp2.columns))


for col_a, col_b in feats:
    df[col_a[:-6] + '_vs'] = df[col_a] - df[col_b]

df.drop(testp1.columns, axis=1, inplace=True)

df.drop(testp2.columns, axis=1, inplace=True)

non_numeric_columns = df.select_dtypes(['object']).columns

df = df.drop(non_numeric_columns, axis=1)

testp1 = df.filter(regex='_P1$', axis=1)

testp2 = df.filter(regex='_P2$', axis=1)

feats = list(zip(testp1.columns, testp2.columns))

for col_a, col_b in feats:
    df[col_a[:-3]] = df[col_a] - df[col_b]

df.drop(testp1.columns, axis=1, inplace=True)

df.drop(testp2.columns, axis=1, inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31725 entries, 0 to 31724
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   match_id                     31725 non-null  int64         
 1   date                         31725 non-null  datetime64[ns]
 2   y                            31725 non-null  int64         
 3   win_percentage_vs            31725 non-null  float64       
 4   bp_factor_vs                 31725 non-null  float64       
 5   first_won_serve_vs           31725 non-null  float64       
 6   second_won_serve_vs          31725 non-null  float64       
 7   ace_probability_vs           31725 non-null  float64       
 8   double_fault_probability_vs  31725 non-null  float64       
 9   aceDf_vs                     31725 non-null  float64       
 10  rank                         31725 non-null  float64       
 11  rank_points                  31725 non-nu

In [10]:
df.to_csv(
    '../db/out/wta_s.csv', index=False)